## 0. 환경 구성

### 1) 라이브러리 설치

In [4]:
%pip install -q langchain langchain-openai langchain_community tiktoken chromadb onnxruntime

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%pip show onnxruntime

Name: onnxruntime
Version: 1.20.1
Summary: ONNX Runtime is a runtime accelerator for Machine Learning models
Home-page: https://onnxruntime.ai
Author: Microsoft Corporation
Author-email: onnxruntime@microsoft.com
License: MIT License
Location: c:\Users\vega2\anaconda3\Lib\site-packages
Requires: coloredlogs, flatbuffers, numpy, packaging, protobuf, sympy
Required-by: chromadb
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install -q langchain langchain-openai langchain_community faiss-cpu

Note: you may need to restart the kernel to use updated packages.


### 2) OpenAI 인증키 설정
https://openai.com/

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


#### RAG 파이프 라인
* Load Data - Text Split - Indexing - Retrieval - Generation

In [3]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

# 1. Load Data
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

# 2️. Text Split
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(documents)
#print(split_docs)

# 3️. Indexing (벡터 저장)
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
# 로컬 파일로 저장
vectorstore.save_local("faiss_index")

# 4️. Retrieval (유사 문서 검색)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
# **질문(쿼리)**에 대해 유사한 문서를 검색하는 역할
retrieved_docs = retriever.invoke("소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
#print(retrieved_docs)

# 5️. Generation (LLM 응답 생성)
llm = ChatOpenAI(model="gpt-4o")
context = "\n\n".join([doc.page_content for doc in retrieved_docs])
#print(context)

response_context = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요? 관련 정보: {context}")
print('context 적용한 결과')
pprint(response_context.content)

response = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
print('context 적용하지 않은 결과')
pprint(response.content)



context 적용한 결과
('소득세법 제12조에 따르면, 비과세소득에 해당하는 소득에 대한 규정은 다음과 같습니다:\n'
 '\n'
 '1. **공익신탁의 이익**: 「공익신탁법」에 따른 공익신탁의 이익은 소득세가 부과되지 않습니다.\n'
 '\n'
 '2. **특정 사업소득**:\n'
 '   - **논ㆍ밭에서의 작물 생산 소득**: 논이나 밭을 작물 생산에 이용하여 발생하는 소득은 비과세됩니다.\n'
 '   - **주택 임대소득**: 1개의 주택을 소유한 사람의 주택임대소득이 비과세됩니다. 단, 제99조에 명시된 기준시가가 12억 원을 '
 '초과하거나 국외에 소재한 주택의 임대소득은 제외됩니다. 또한, 해당 과세기간에 총수입금액이 2천만 원 이하인 경우에도 비과세됩니다. 이 '
 '경우, 주택 수의 계산 및 주택임대소득의 산정 등에 관한 구체적인 사항은 대통령령에 따릅니다.\n'
 '   - **농어가부업소득**: 이를 포함한 일정한 농업 관련 부업소득도 대통령령으로 정하는 바에 따라 비과세됩니다.\n'
 '   - **전통주의 제조 소득**: 전통주 제조에서 발생하는 소득도 대통령령으로 정하여 비과세됩니다.\n'
 '   - **임지의 임목 벌채 또는 양도 소득**: 조림기간이 5년 이상인 임지에서 발생하는 임목의 벌채나 양도로 인한 소득이 연 '
 '600만 원 이하인 경우 비과세됩니다. 이 경우의 조림기간 및 세액의 계산 등에 필요한 사항은 대통령령에 따릅니다.\n'
 '   - **작물재배업 소득**: 대통령령으로 정하는 작물재배업에서 발생하는 소득이 비과세됩니다.\n'
 '\n'
 '이 규정들은 소득세의 과세 대상에서 제외되며, 구체적인 세부 사항은 관련 대통령령에 따릅니다. 각 항목에 대한 자세한 내용은 관련 '
 '법령이나 규정을 통해 확인할 수 있습니다.')
context 적용하지 않은은 결과
('소득세법에서 비과세 소득은 세금을 부과하지 않는 소득을 의미하며, 국가마다 규정이 다를 수 있습니다. 일반적으로 비과세 소득에는 다음과 '
 

### 개선한 Source

In [2]:

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

# 1. 데이터 로드 (기존과 동일)
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

# 2. 텍스트 분할 개선
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 크기 증가
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""],  # 자연스러운 분할을 위한 구분자
    length_function=len,
    is_separator_regex=False,
)
split_docs = splitter.split_documents(documents)

# 3. 인덱싱 (벡터 저장)
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
vectorstore.save_local("faiss_index")

# 4. 검색 개선
retriever = vectorstore.as_retriever(
    search_type="mmr",  # 최대 다양성 검색
    search_kwargs={"k": 5, "fetch_k": 10}  # 더 많은 결과 검색
)

# 5. 프롬프트 엔지니어링
def generate_prompt(query, context):
    return f"""다음은 소득세법 비과세소득 관련 조항입니다. 문맥을 고려하여 질문에 답변하세요.

[관련 조항]
{context}

[질문]
{query}

[답변 요구사항]
- 비과세소득 유형을 계층적으로 구분하여 설명
- 각 항목별 구체적인 조건 명시
- 법조문의 항, 호, 목 번호를 포함
- 500자 이내로 간결하게 요약"""

# 검색 및 응답 생성
query = "소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?"
retrieved_docs = retriever.invoke(query)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

llm = ChatOpenAI(model="gpt-4o", temperature=0.3)  # 창의성 낮춤
response = llm.invoke(generate_prompt(query, context))

print('개선된 결과:')
pprint(response.content)

: 